In [ ]:
import numpy as np, os 
import pandas as pd, gc 
from tqdm import tqdm
from sklearn.metrics import accuracy_score

In [ ]:
VERSION = "d14" # 主题词版本

SAVE_TOPWORD_NUM = 50 # 类型词 数量（每类）

RUN_TYPE_KEYWORDS = False # 是否重新跑 “类型词“

USE_STOPWODS2 = True
HANDLE_ERRER = False # 目前还待修改的地方

In [ ]:
train_df = pd.read_csv('../input/feedback-prize-2021/train.csv')
print( train_df.shape )
train_df.head()

In [ ]:
discourse_types = ['Lead', 'Position', 'Claim', 'Counterclaim','Rebuttal','Evidence', 'Concluding Statement']

## 文本清理

In [ ]:
from collections import defaultdict
from nltk.corpus import stopwords as nltk_stopwords
from tqdm import tqdm
from ast import literal_eval

In [ ]:
import re
import pandas as pd
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import string
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords as nltk_stopwords

# 目地：句子处理，句子特征挖掘

def get_stopwords(): 
    # 备注： 大词表不好： 可能词表中 有大量词语 需要保留，如because、against等等 

    stop_words = set(nltk_stopwords.words('english')) # 小词表：不到200词
    # print(len(stop_words),stop_words)
    remain_words = ['should','can',"can't","couldn","couldn't","must","mustn't","mustn","need","needn't", "wouldn","wouldn't","shouldn","shouldn't","mightn't","against","because"]
    for w in remain_words:
        if w in stop_words:
            stop_words.remove(w)
    # print(len(stop_words),stop_words)

    customized_stopwords = ['student', 'people', 'make', 'school', 'teacher', 'be', 'college']
    for token in customized_stopwords:
        stop_words.add(token)

    print(f'There are {len(stop_words)} default stop-words inlcuding customization.\n')
    print("-"*100)
    # print(f'There are {len(stopwords)} stop-words after customization.')
    return stop_words


def split_charachers(text):
    result_list = re.split('(。|！|\!|\.|？|\?|,|，)| \n', text)
    return result_list

def remove_charachers(text):
    _text = re.sub(r'(。|！|\!|\.|？|\?|,|\n)', " ",text)
    return _text

def sent2rawwords(sent):
    #  获取单词的词性
    def get_wordnet_pos(tag):
        if tag.startswith('J'):
            return wordnet.ADJ
        elif tag.startswith('V'):
            return wordnet.VERB
        elif tag.startswith('N'):
            return wordnet.NOUN
        elif tag.startswith('R'):
            return wordnet.ADV
        else:
            return None
    tagged_sent = pos_tag(sent)  # 获取词性
    wnl = WordNetLemmatizer()
    lemmas_sent = []
    for tag in tagged_sent:
        wordnet_pos = get_wordnet_pos(tag[1]) or wordnet.NOUN
        lemmas_sent.append(wnl.lemmatize(tag[0], pos=wordnet_pos))  # 词形还原
    return lemmas_sent


def clean_text(sentence, stopwords, lemma = False):
    sentence = sentence.lower()
    # 1. 使用EduNLP
    # tokenzier = PureTextTokenizer()
    # token_item = tokenzier([sentence])
    # text = [word for word in next(token_item) if word not in stopwords]
    
    # 2. 使用nltk 
    # （1）符号清洗 #（注意是否需要修改）
    sent = remove_charachers(sentence) # V2 
    # print("check 1:::", sentence)
    text_raw_list = sent.split() # Vb # 不用strip()
    # print("check 2:::", text_raw_list)
    # （2）. 词形还原
    if lemma:
        text_raw_list = sent2rawwords(text_raw_list) # V1-5
    # print("check 3:::", text_raw_list)
    # print("text_raw_list : ",text_raw_list)
    # （3）. 停用词 过滤
    text = [word for word in text_raw_list if word not in stopwords]
    if "student" in text:
        raise ValueError("What the fuck???!!!")
    # print("check 4:::", text)
    return " ".join(text).strip()


# 词形还原2（效果不好：不使用）
def stem_text(sentence):
    ps = PorterStemmer()
    sent = []
    for word in sentence:
        sent.append(ps.stem(word))
    return " ".join(sent)

# if STEM_TEXT:
#     text = [t.split() for t in train_text_df["text"].tolist()]

#     stemmed_text = []
#     ps = PorterStemmer()
#     for sentence in tqdm(text):
#         stemmed_text.append(stem_text(sentence))

#     train_text_df["text_stemmed"] = [" ".join(l) for l in stemmed_text]
#     train_text_df.to_csv(f"../../input/feedback-prize-2021/train_essay_stemmed_{DATA_VERSION}.csv", index=False, sep=',')
# else:
#     train_text_df = pd.read_csv(f"../../input/feedback-prize-2021/train_essay_stemmed_{DATA_VERSION}.csv")



# if CLEAN_TEXT:
#     stopwords = get_stopwords()
#     texts = []
#     for sentence in tqdm(train_text_df["text_stemmed"].values):
#         sentence = clean_text(sentence, stopwords)
#         if not sentence:
#             sentence = ""
#         texts.append(sentence)
#     train_text_df["text_stemmed_cleaned"] = texts
#     train_text_df.to_csv(f"./train_essay_stemmed_cleaned_{DATA_VERSION}.csv", index=False, sep=',')
# else:
#     train_text_df = pd.read_csv(f"./train_essay_stemmed_cleaned_{DATA_VERSION}.csv")

In [ ]:
# # 测试词性还原
# test_sent = "In mars, If the Students asks both his Friends and his doctors, he wouldn't able to use his judgemental skills to determing which choice will be best for him in the long run."

# _s1 = stem_text(test_sent.split()) # 不够智能，词形还原有时错误还原
# _s2 = " ".join(sent2rawwords(test_sent.lower().split())) # 不够智能，词形还原有时错误还原
# stopwords = get_stopwords()
# _s3 = clean_text(test_sent, stopwords, lemma=True)

# print("-"*50,"\n",_s1)
# print("-"*50,"\n",_s2)
# print("-"*50,"\n",_s3)
# -------------------------------------------------- 
#  in mars, if the student ask both hi friend and hi doctors, he wouldn't abl to use hi judgement skill to determ which choic will be best for him in the long run.
# -------------------------------------------------- 
#  in mars, if the student ask both his friend and his doctors, he wouldn't able to use his judgemental skill to determing which choice will be best for him in the long run.
# -------------------------------------------------- 
#  mars, ask friend doctors, wouldn't able use judgemental skill determing choice best long run.

In [ ]:
def get_frequent_stopwords(corpus, _stop_words):
    dic= defaultdict(int)
    for word in corpus:
        if word in _stop_words:
            dic[word]+=1

    # getting the top 20 most frequent stop words        
    top=sorted(dic.items(), key=lambda x:x[1],reverse=True)[:20]
    
    return top, dic

corpus = []
train_df["words"] = train_df["discourse_text"].apply(lambda x: x.split())
for item in tqdm(train_df["words"].values):
    # print(item)
    corpus.extend(item)
# corpus

_stop_words = get_stopwords()
top, dic = get_frequent_stopwords(corpus, _stop_words)

print(f'Total number of stop words in spacy: {len(_stop_words)}')
print(f'Total number of stop words in the given text data which are part of the spacy\'s stop words list: {len(dic)}')
print(f"top words: {top}")

import matplotlib.pyplot as plt
# top 20 most frequent stop words in our corpus:
plt.figure(figsize = (15,4))
x,y=zip(*top)
plt.bar(x,y, color='goldenrod')

## 提取类型关键词

In [ ]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# topic_set

In [ ]:
# ------------------------------------ 加载主题词 -------------------------------------- #
with open(f'../input/myfeedbackdata/predata/topic/topic_words_{VERSION}.pkl', 'rb') as pkl_file:
  all_topic_words = pickle.load(pkl_file)

topic_set = [] # [(word, weight)]
topic_num = 0
for _list in all_topic_words:
  topic_set.extend(_list)
  topic_num +=1

topic_set = set(topic_set)

topic_word_2_weight = {k:v for k,v in topic_set} # 无需不考虑重复的主题词的权值
topic_word_set = set(topic_word_2_weight.keys())

print(f"There are {len(topic_word_2_weight)} topic words in {topic_num} topic !")

# ---------------------------------- 处理类型词 ---------------------------------------- #
MY_STOPWORDS = get_stopwords()  # 停用词

def get_tfdif_keywords(text_list):
  from sklearn.feature_extraction.text import TfidfVectorizer
  text_list = ["i am  a fucking student"] + text_list
  for text in text_list:
    # if 'car' in text.split():
    #   raise ValueError("3 car Something wrong!, car in still in _text_list_f")
    if HANDLE_ERRER and "student" in text.split():
        raise ValueError("3 student Something wrong!, car in still in _text_list_f")

  _stop_words = "english" if USE_STOPWODS2 else None
  _tfidf = TfidfVectorizer(stop_words=_stop_words, binary=True, max_features=25_000) # stop_words='english'
  text_embeddings = _tfidf.fit_transform( text_list ).toarray()
  print("shape of text_embeddings",text_embeddings.shape)
  print("len of features",len(_tfidf.get_feature_names()))
  mm = np.mean( text_embeddings, axis=0 )
  ii = np.argsort(mm)[-SAVE_TOPWORD_NUM:][::-1].tolist()
  # print("test ii:::",ii)
  # print(ii)
  top_words = [(_tfidf.get_feature_names()[i], mm[i]) for i in ii]
  # print(_tfidf.get_feature_names())
  # if 'car' in _tfidf.get_feature_names():
  #   raise ValueError("4 car Something wrong!, car in still in _text_list_f")
  if HANDLE_ERRER and  "student" in _tfidf.get_feature_names():
    raise ValueError("4 student Something wrong!, car in still in _text_list_f")

  print(f"{SAVE_TOPWORD_NUM} top words: \n",*top_words[:15])
  return top_words

def clean_discourse(text):
  # 过滤停用词
  text = clean_text(text, MY_STOPWORDS, lemma=True)
  # 过滤主题词
  text_list = text.strip().split()
  # for word in text_list:
  #   if word not in topic_word_set:
  #     text_list.append(word)
    
  _text_list_f = [word for word in text_list if word not in topic_word_set]
  if HANDLE_ERRER and  'car' in _text_list_f or "student" in _text_list_f:
    raise ValueError("2 Something wrong!, car in still in _text_list_f")
  return " ".join(_text_list_f)

def get_keywords_by_type(_type):

  
  text_list = train_df.loc[train_df["discourse_type"] == _type, "discourse_text"].values
  text_list2 = []
  for text in  tqdm(text_list):
    _text = clean_discourse(text)
    if HANDLE_ERRER and 'car' in _text.split() or "student" in _text.split():
      raise ValueError("1 Something wrong!, car in still in _text_list_f")
    text_list2.append(_text)
  text_list2 = ["i am a fucking student"] + text_list2
  print(text_list2[:5])
  top_words = get_tfdif_keywords(text_list2)    

  return top_words

In [ ]:
# 测试
text_list2 = []
text = "the fucking students,and student ,the cars, is in the outside, you are good!"
text = clean_discourse(text)
print(text.split())
text_list2.append(text)

top_wprds = get_tfdif_keywords(text_list2)
top_wprds

In [ ]:
XX = np.array([[1,2,3,4]])
print(XX)
print(XX.flatten())


In [ ]:
# # 测试 TFIDF
# from sklearn.feature_extraction.text import TfidfVectorizer
# corpus = [
#      'This is the first document.',
#      'This document is the second document.',
#      'And this is the third one.',
#      'Is this the first document?',
#  ]
# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(corpus).toarray()
# pre = vectorizer.get_feature_names()
# print(pre)
# print(X.shape)

# mm = np.mean( X, axis=0 )
# print(mm)
# x1 = np.argsort(mm)
# print("x1",type(x1),x1.shape,x1)
# # print("x1", np.array(x1).flatten())
# # print("x1", x1.tolist()[0])

# # x2 = x1[-3:][::1]
# # print("x2", x2)

# ii = np.argsort(mm)[-3:][::-1].tolist()
# print(ii)
# # print(ii)
# top_words = [(vectorizer.get_feature_names()[i], mm[i]) for i in ii]
# print(top_words)
# print(vectorizer.vocabulary_["this"])

# # 测试
# get_tfdif_keywords(corpus)

In [ ]:
# 测试，有过滤不彻底的问题！！！！！！
# tt = get_keywords_by_type("Lead")

In [ ]:
if RUN_TYPE_KEYWORDS: # 处理 类型词
  all_type_words = dict()
  for _type in discourse_types:
    all_type_words[_type] = get_keywords_by_type(_type)
  print("Save the results.....")
  # 保存 类型词
  with open(f'type_words_{VERSION}.pkl', 'wb') as output:
    pickle.dump(all_type_words, output)

else: # 直接加载 类型词
  with open(f'../input/myfeedbackdata/predata/type/type_words_{VERSION}.pkl', 'rb') as pkl_file:
    all_type_words = pickle.load(pkl_file)
  

In [ ]:
# 查看类型词语
for k,v in all_type_words.items():
  print(f"---------{k}--------------")
  tt = [t for t, _ in  v]
  print(tt)

In [ ]:
# The wrong res
# ---------Lead--------------
# ['think', 'car', 'advice', 'ask', 'like', 'good', 'time', 'way', 'help', 'use', 'know', 'want', 'class', 'home', 'person', 'need', 'electoral', 'thing', 'phone', 'learn', 'online', 'school', 'driverless', 'summer', 'say', 'opinion', 'work', 'face', 'activity', 'life', 'technology', 'new', 'project', 'drive', 'idea', 'world', 'learning', 'come', 'cell', 'vote', 'attend', 'talk', 'able', 'multiple', 'day', 'extracurricular', 'change', 'year', 'president', 'really']
# ---------Position--------------
# ['think', 'good', 'car', 'believe', 'use', 'electoral', 'idea', 'opinion', 'class', 'benefit', 'phone', 'help', 'driverless', 'project', 'multiple', 'community', 'able', 'home', 'activity', 'advice', 'ask', 'service', 'policy', 'summer', 'participate', 'change', 'agree', 'vote', 'extracurricular', 'attend', 'choice', 'person', 'cell', 'way', 'online', 'seek', 'great', 'venus', 'reason', 'design', 'limit', 'face', 'popular', 'like', 'thing', 'usage', 'technology', 'allow', 'designed', 'drive']
# ---------Claim--------------
# ['help', 'good', 'car', 'time', 'reason', 'work', 'think', 'learn', 'need', 'class', 'use', 'like', 'electoral', 'thing', 'phone', 'know', 'want', 'way', 'vote', 'able', 'different', 'home', 'advice', 'project', 'activity', 'community', 'kid', 'opinion', 'venus', 'person', 'allow', 'ask', 'new', 'online', 'drive', 'life', 'driverless', 'lot', 'school', 'idea', 'feel', 'state', 'president', 'cell', 'say', 'technology', 'benefit', 'face', 'sport', 'cause']
# ---------Counterclaim--------------
# ['say', 'think', 'electoral', 'argue', 'class', 'time', 'good', 'car', 'like', 'work', 'online', 'home', 'project', 'phone', 'want', 'vote', 'thing', 'know', 'use', 'believe', 'help', 'need', 'way', 'learn', 'reason', 'idea', 'hand', 'bad', 'kid', 'benefit', 'state', 'yes', 'driverless', 'alien', 'face', 'able', 'learning', 'design', 'drive', 'president', 'problem', 'summer', 'understand', 'attend', 'community', 'life', 'cell', 'designed', 'service', 'school']
# ---------Rebuttal--------------
# ['time', 'think', 'help', 'good', 'thing', 'like', 'vote', 'work', 'need', 'way', 'know', 'want', 'class', 'learn', 'use', 'phone', 'project', 'car', 'true', 'really', 'online', 'wrong', 'say', 'electoral', 'life', 'home', 'able', 'happen', 'benefit', 'problem', 'reason', 'state', 'come', 'kid', 'school', 'believe', 'lot', 'idea', 'change', 'mean', 'president', 'best', 'feel', 'day', 'try', 'actually', 'design', 'community', 'allow', 'popular']
# ---------Evidence--------------
# ['help', 'like', 'time', 'good', 'know', 'want', 'work', 'think', 'need', 'thing', 'say', 'car', 'way', 'use', 'class', 'example', 'able', 'state', 'person', 'phone', 'vote', 'ask', 'learn', 'home', 'life', 'day', 'school', 'lot', 'really', 'friend', 'come', 'drive', 'tell', 'kid', 'try', 'feel', 'electoral', 'new', 'different', 'advice', 'look', 'reason', 'cause', 'president', 'best', 'talk', 'bad', 'grade', 'sport', 'project']
# ---------Concluding Statement--------------
# ['good', 'think', 'help', 'way', 'time', 'conclusion', 'need', 'thing', 'car', 'like', 'want', 'use', 'life', 'know', 'reason', 'learn', 'idea', 'opinion', 'work', 'electoral', 'change', 'ask', 'best', 'phone', 'believe', 'advice', 'say', 'able', 'class', 'person', 'benefit', 'new', 'school', 'great', 'choice', 'really', 'vote', 'world', 'multiple', 'community', 'different', 'future', 'lot', 'come', 'allow', 'activity', 'driverless', 'feel', 'policy', 'drive']

#------------------ 经过处理后-------------------------#

# ---------Lead--------------
# ['imagine', 'question', 'big', 'everyday', 'start', 'wonder', 'consider', 'mean', 'actually', 'sure', 'debate', 'society', 'write', 'instead', 'issue', 'dont', 'family', 'mind', 'student', 'end', 'certain', 'daily', 'designed', 'teacher', 'sound', 'fact', 'difficult', 'usually', 'america', 'letter', 'negative', 'public', 'father', 'course', 'argue', 'making', 'type', 'possible', 'positive', 'disagree', 'probably', 'thought', 'today', 'order', 'second', 'past', 'explain', 'driverless', 'chance', 'real']
# ---------Position--------------
# ['designed', 'disagree', 'shouldn', 'student', 'yes', 'personally', 'teacher', 'favor', 'suggest', 'actually', 'strongly', 'instead', 'dont', 'development', 'useful', 'venus', 'fact', 'argue', 'consider', 'positive', 'face', 'claim', 'worth', 'rid', 'explain', 'mandatory', 'big', 'number', 'smart', 'matter', 'capacity', 'driveless', 'america', 'start', 'negative', 'wouldn', 'self', 'position', 'belive', 'write', 'develop', 'society', 'type', 'im', 'greatly', 'collage', 'possible', 'mean', 'provide', 'completely']
# ---------Claim--------------
# ['chance', 'second', 'actually', 'big', 'dont', 'family', 'mean', 'skill', 'provide', 'wouldn', 'start', 'issue', 'improve', 'certain', 'student', 'fact', 'instead', 'lastly', 'shouldn', 'increase', 'finally', 'sure', 'self', 'mind', 'outcome', 'meet', 'lose', 'prevent', 'ability', 'designed', 'worry', 'spend', 'responsibility', 'possible', 'freedom', 'end', 'secondly', 'probably', 'avoid', 'case', 'health', 'open', 'matter', 'type', 'receive', 'smart', 'final', 'outside', 'test', 'public']
# ---------Counterclaim--------------
# ['argue', 'yes', 'designed', 'disagree', 'waste', 'teacher', 'student', 'argument', 'dont', 'course', 'sure', 'case', 'actually', 'claim', 'wouldn', 'fact', 'probably', 'avoid', 'true', 'public', 'possible', 'outcome', 'certain', 'issue', 'mean', 'big', 'chance', 'sound', 'theorist', 'negative', 'lazy', 'wont', 'conspiracy', 'family', 'lose', 'marking', 'haze', 'start', 'positive', 'receive', 'worry', 'provide', 'cheat', 'downside', 'instead', 'skill', 'opponent', 'question', 'flaw', 'cool']
# ---------Rebuttal--------------
# ['true', 'mean', 'actually', 'fact', 'case', 'dont', 'end', 'big', 'wouldn', 'chance', 'matter', 'reality', 'start', 'thats', 'worth', 'sure', 'real', 'simply', 'skill', 'student', 'disagree', 'shouldn', 'instead', 'public', 'easily', 'certain', 'possible', 'designed', 'trust', 'far', 'leave', 'issue', 'self', 'course', 'realize', 'provide', 'wish', 'argument', 'avoid', 'yes', 'completely', 'positive', 'false', 'teacher', 'question', 'hurt', 'freedom', 'flaw', 'negative', 'probably']
# ---------Evidence--------------
# ['mean', 'start', 'dont', 'wouldn', 'big', 'family', 'instead', 'end', 'chance', 'actually', 'sure', 'probably', 'test', 'leave', 'fact', 'lose', 'certain', 'mind', 'spend', 'question', 'worry', 'case', 'house', 'matter', 'skill', 'group', 'result', 'type', 'second', 'thats', 'issue', 'far', 'self', 'possible', 'public', 'small', 'number', 'wont', 'trust', 'old', 'pass', 'young', 'large', 'hold', 'outside', 'order', 'stuff', 'accord', 'include', 'real']
# ---------Concluding Statement--------------
# ['end', 'overall', 'chance', 'dont', 'shouldn', 'big', 'sure', 'instead', 'consider', 'improve', 'fact', 'matter', 'student', 'mean', 'provide', 'start', 'skill', 'actually', 'family', 'mind', 'possible', 'wouldn', 'designed', 'finally', 'thats', 'positive', 'worth', 'conclude', 'leave', 'disagree', 'self', 'trust', 'society', 'question', 'open', 'remember', 'yes', 'smart', 'outcome', 'real', 'meet', 'teacher', 'issue', 'increase', 'worry', 'lose', 'public', 'probably', 'useful', 'effect']


#------------------ 最终处理后（TFIDF统计时使用停用词）（d14版本）-------------------------#
# ---------Lead--------------
# ['imagine', 'question', 'big', 'everyday', 'start', 'wonder', 'consider', 'mean', 'actually', 'sure', 'debate', 'society', 'write', 'instead', 'issue', 'family', 'dont', 'mind', 'end', 'student', 'certain', 'daily', 'designed', 'teacher', 'sound', 'fact', 'difficult', 'father', 'usually', 'america', 'negative', 'letter', 'public', 'course', 'force', 'argue', 'making', 'possible', 'type', 'positive', 'disagree', 'probably', 'thought', 'today', 'order', 'past', 'explain', 'driverless', 'matter', 'real']
# ---------Position--------------
# ['designed', 'disagree', 'shouldn', 'student', 'yes', 'personally', 'teacher', 'favor', 'suggest', 'actually', 'strongly', 'instead', 'dont', 'force', 'development', 'useful', 'fact', 'venus', 'argue', 'consider', 'positive', 'face', 'claim', 'worth', 'rid', 'explain', 'mandatory', 'big', 'number', 'smart', 'matter', 'capacity', 'america', 'driveless', 'start', 'negative', 'wouldn', 'self', 'belive', 'position', 'write', 'develop', 'society', 'im', 'type', 'greatly', 'collage', 'possible', 'provide', 'mean']
# ---------Claim--------------
# ['actually', 'big', 'dont', 'family', 'mean', 'skill', 'provide', 'wouldn', 'start', 'issue', 'improve', 'certain', 'student', 'fact', 'instead', 'lastly', 'increase', 'shouldn', 'sure', 'finally', 'self', 'mind', 'meet', 'outcome', 'lose', 'prevent', 'ability', 'designed', 'spend', 'worry', 'responsibility', 'force', 'possible', 'freedom', 'end', 'secondly', 'avoid', 'case', 'probably', 'open', 'health', 'type', 'matter', 'receive', 'smart', 'final', 'public', 'outside', 'real', 'test']
# ---------Counterclaim--------------
# ['argue', 'yes', 'designed', 'disagree', 'waste', 'teacher', 'student', 'argument', 'dont', 'course', 'sure', 'case', 'actually', 'claim', 'wouldn', 'fact', 'avoid', 'probably', 'true', 'public', 'possible', 'outcome', 'issue', 'certain', 'big', 'mean', 'sound', 'theorist', 'negative', 'lazy', 'wont', 'lose', 'family', 'conspiracy', 'marking', 'start', 'haze', 'positive', 'receive', 'worry', 'provide', 'cheat', 'downside', 'instead', 'skill', 'opponent', 'flaw', 'question', 'cool', 'difficult']
# ---------Rebuttal--------------
# ['true', 'mean', 'actually', 'fact', 'case', 'dont', 'end', 'big', 'wouldn', 'matter', 'reality', 'thats', 'start', 'worth', 'simply', 'sure', 'real', 'skill', 'student', 'disagree', 'shouldn', 'public', 'instead', 'easily', 'force', 'certain', 'possible', 'designed', 'far', 'self', 'leave', 'issue', 'course', 'realize', 'provide', 'argument', 'wish', 'avoid', 'yes', 'completely', 'positive', 'false', 'teacher', 'hurt', 'question', 'freedom', 'flaw', 'negative', 'probably', 'prevent']
# ---------Evidence--------------
# ['mean', 'start', 'dont', 'wouldn', 'big', 'family', 'end', 'instead', 'actually', 'sure', 'probably', 'test', 'fact', 'leave', 'lose', 'certain', 'mind', 'spend', 'question', 'worry', 'case', 'house', 'matter', 'group', 'skill', 'type', 'thats', 'result', 'issue', 'far', 'possible', 'self', 'public', 'force', 'small', 'number', 'young', 'wont', 'old', 'pass', 'hold', 'large', 'outside', 'order', 'stuff', 'include', 'accord', 'real', 'explain', 'provide']
# ---------Concluding Statement--------------
# ['end', 'overall', 'dont', 'sure', 'shouldn', 'big', 'instead', 'consider', 'improve', 'fact', 'matter', 'student', 'mean', 'provide', 'start', 'actually', 'skill', 'family', 'mind', 'possible', 'force', 'wouldn', 'designed', 'finally', 'thats', 'positive', 'worth', 'conclude', 'leave', 'disagree', 'self', 'society', 'open', 'question', 'remember', 'yes', 'smart', 'outcome', 'real', 'meet', 'teacher', 'issue', 'increase', 'worry', 'lose', 'public', 'useful', 'probably', 'effect', 'ability']

#------------------ (TFIDF统计时不用停用词)-------------------------#
# ---------Lead--------------
# ['can', 'many', 'one', 'should', 'would', 'get', 'go', 'ever', 'take', 'because', 'could', 'well', 'may', 'even', 'something', 'also', 'give', 'system', 'others', 'someone', 'keep', 'see', 'whether', 'everyone', 'sometimes', 'every', 'around', 'always', 'become', 'might', 'call', 'however', 'much', 'find', 'imagine', 'still', 'least', 'big', 'question', 'seem', 'everyday', 'start', 'although', 'consider', 'two', 'mean', 'wonder', 'part', 'actually', 'must']
# ---------Position--------------
# ['should', 'would', 'can', 'one', 'many', 'because', 'keep', 'get', 'go', 'take', 'could', 'well', 'system', 'designed', 'least', 'someone', 'disagree', 'shouldn', 'student', 'give', 'may', 'even', 'against', 'something', 'also', 'others', 'much', 'everyone', 'although', 'yes', 'see', 'teacher', 'personally', 'favor', 'suggest', 'strongly', 'must', 'actually', 'every', 'become', 'the', 'however', 'always', 'instead', 'dont', 'might', 'find', 'development', 'force', 'though']
# ---------Claim--------------
# ['can', 'because', 'would', 'get', 'also', 'could', 'should', 'one', 'take', 'go', 'give', 'many', 'another', 'may', 'might', 'something', 'see', 'even', 'less', 'well', 'first', 'others', 'someone', 'much', 'system', 'keep', 'find', 'show', 'everyone', 'still', 'become', 'around', 'always', 'put', 'big', 'call', 'actually', 'dont', 'skill', 'every', 'provide', 'family', 'mean', 'last', 'sometimes', 'without', 'wouldn', 'already', 'start', 'issue']
# ---------Counterclaim--------------
# ['may', 'would', 'because', 'can', 'might', 'many', 'argue', 'could', 'get', 'although', 'take', 'also', 'should', 'go', 'one', 'even', 'though', 'however', 'still', 'yes', 'well', 'others', 'see', 'keep', 'system', 'give', 'much', 'designed', 'something', 'disagree', 'sometimes', 'another', 'waste', 'everyone', 'teacher', 'someone', 'student', 'side', 'less', 'always', 'argument', 'dont', 'sure', 'course', 'call', 'seem', 'claim', 'case', 'actually', 'become']
# ---------Rebuttal--------------
# ['would', 'can', 'because', 'however', 'get', 'should', 'take', 'still', 'could', 'go', 'one', 'many', 'also', 'even', 'may', 'well', 'true', 'give', 'something', 'much', 'others', 'see', 'system', 'though', 'always', 'mean', 'everyone', 'actually', 'might', 'although', 'find', 'fact', 'show', 'case', 'every', 'keep', 'put', 'less', 'that', 'dont', 'enough', 'another', 'end', 'without', 'someone', 'around', 'big', 'wouldn', 'matter', 'since']
# ---------Evidence--------------
# ['can', 'get', 'would', 'because', 'go', 'could', 'also', 'one', 'take', 'even', 'should', 'give', 'many', 'something', 'see', 'may', 'might', 'someone', 'well', 'much', 'another', 'show', 'always', 'around', 'still', 'find', 'keep', 'mean', 'others', 'put', 'system', 'everyone', 'every', 'become', 'start', 'dont', 'back', 'big', 'less', 'without', 'sometimes', 'call', 'wouldn', 'first', 'instead', 'anything', 'end', 'family', 'actually', 'since']
# ---------Concluding Statement--------------
# ['should', 'can', 'because', 'would', 'get', 'one', 'many', 'go', 'take', 'could', 'also', 'give', 'see', 'well', 'even', 'something', 'may', 'keep', 'much', 'others', 'everyone', 'always', 'system', 'someone', 'might', 'still', 'find', 'put', 'less', 'show', 'become', 'every', 'around', 'end', 'overall', 'that', 'never', 'dont', 'without', 'big', 'sure', 'instead', 'shouldn', 'next', 'consider', 'least', 'though', 'please', 'improve', 'fact']

In [ ]:
import matplotlib.pyplot as plt

SHOW_NUM = 20
x=range(0, SHOW_NUM)

fig, ax = plt.subplots( len(discourse_types) , 1, figsize = (20, 50))
fig.suptitle(F'Top {SHOW_NUM} Bigrams of the TF-IDF', fontsize= 18)


for i, _type in enumerate(discourse_types):
  _pairs = all_type_words[_type]
  
  sorted(_pairs,key=(lambda x:x[1]),reverse=True)
  
  y, words = [], []
  for k,v in _pairs[:SHOW_NUM]:
    y.append(v)
    words.append(k)

  print(len(y))
  ax[i].plot(x, y, 'bo')
  ax[i].set_title(_type, fontsize= 14)
  ax[i].set_xticks(x)
  ax[i].set_xticklabels(words, rotation='vertical', fontsize=10)

fig.subplots_adjust(hspace=1.5)
plt.show()